In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

In [2]:
device = {
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'  
}   
print(device)

{'cuda'}


In [3]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should show your GPU name


True
NVIDIA GeForce RTX 3050


In [4]:
import torch
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.6.0+cu124
CUDA version: 12.4
CUDA available: True


In [5]:
# HyperParameters
batch_size = 32
block_size = 128
max_iters = 20000
learning_rate = 3e-4
eval_iters = 200
n_embd = 256
n_head = 8
n_layer = 6
dropout = 0.1


In [6]:
chars = ""
with open('../wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)    

In [10]:
with open('../wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [8]:
chars = ""
with open("vocab.txt" , 'r', encoding='utf-8') as f:
    f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

FileNotFoundError: [Errno 2] No such file or directory: 'vocab.txt'

In [11]:
# Suppose chars is a list or string of unique characters
# e.g., chars = sorted(list(set(text)))

string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Encode: string -> list of integers
encode = lambda s: [string_to_int[c] for c in s]

# Decode: list of integers -> string
decode = lambda l: ''.join(int_to_string[i] for i in l)

# Convert encoded data to a tensor
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [12]:
def get_random_chunk(split):
    filename = "../wizard_of_oz.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)
            
            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size-1)
            
            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long )
            
        return data
    

In [13]:
def get_batch(split) :
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i : i + batch_size] for i in ix])
    y = torch.stack([data[i + 1 : i + batch_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
class Head(nn.Module ) :
    """ One head of self-attention """
    
    def __init__(self, head_size) : 
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False )
        self.query = nn.Linear(n_embd, head_size, bias=False )
        self.value = nn.Linear(n_embd, head_size, bias=False )
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size )))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input of size (batch ,time-step, channels)
        # output of size (batch, time-step, head size)
        
        B, T, C = x.shape
        k = self.key(x)    # (B, T , hs)
        q = self.query(x)  # (B, hs, T)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, 1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[ : T, : T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        #perform the weighted aggregation of the values
        v = self.values(x) # (B, T, hs)
        out = wei @ v
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size) :
        super().__init__()
        self.heads = nn.ModuleList(Head(head_size) for _ in range(num_heads))
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        #  (B, T, F) -> ( B, T , [h1, h1, h1, h1, h2, h2, h2, h2, h3 ... ])
        out = self,dropout(self.proj(out))
        return out
    

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed y a non-linearity """
    
    def __init__(self, n_embd) : 
        super().__init__()
        self.net = nn.Sequencial(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
    def forward(self, x) :
        return self.net(x)

In [ ]:
class Block(nn.Module) :
    """" Transformer block : communication followed by computation """
    
    def __init__(self, n_embd, n_head):
        #n_embd: embedding dimension, n_head : number of heads we'd like to be 
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x) :
        y = self.sa(x)
        x = self.ln1( x + y)
        y = self.ffwd(x)
        x = self.ln2( x + y)
        return x

In [ ]:
class GPTLanguageModel(nn.Module):
    
    def __init__(self, vocab_size) : 
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequencial(* [Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self.init_weights)
        
    def __init_weights(self, module) :
        if isinstance(module, nn.Linear) :
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None :
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding) :
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            
    def forward(self, index, targets=None) : 
        B, T = index.shape
        
        # index and targets both B, T tesors of integers
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arrange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T , C)
        x = self.ln_f(x) # (B, T , C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None : 
            loss = None
        else : 
            B, T , C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, index, max_new_tokens ) :
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens) : 
            # Crop index to the Last block size tokens
            index_cond = index[ : , -block_size : ]
            # get the predictions
            logits, loss = self.forward(index_cond)
            #focus only on the loast time step
            probs = logits[ : , -1, :] # Becomes ( B , C)
            # apply softmax to get probablities
            probs = F.softmax( logits, dim=-1) # (B, C)
            # Sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # ( B, 1)
            # appened sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, t + 1)
        return index
    


AttributeError: cannot assign module before Module.__init__() call

In [ ]:
model = GPTLanguageModel(vocab_size)

m = model.to(device)

AttributeError: cannot assign module before Module.__init__() call

In [14]:


class Head(nn.Module):
    """One head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # tril is a lower-triangular mask for causal self-attention
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)                     # (B, T, hs)
        q = self.query(x)                   # (B, T, hs)

        # attention weights
        wei = (q @ k.transpose(-2, -1)) * (k.shape[-1] ** -0.5)  # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # weighted sum of values
        v = self.value(x)                   # (B, T, hs)
        out = wei @ v                       # (B, T, hs)
        return out


class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Concatenate head outputs along the channel dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """A simple linear layer followed by a non-linearity"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# ───────────────────────────────────────────────
# GPT Model
# ───────────────────────────────────────────────

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        # token + position embeddings (device-safe)
        tok_emb = self.token_embedding_table(index)  # (B, T, C)
        pos_emb = self.position_embedding_table(
            torch.arange(T, device=index.device)
        )                                           # (T, C)
        x = tok_emb + pos_emb                       # (B, T, C)

        # pass through transformer blocks
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)                    # (B, T, vocab_size)

        # compute loss if targets given
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(
                logits.view(B * T, C),
                targets.view(B * T)
            )

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = index[:, -block_size:]
            logits, _ = self.forward(idx_cond)
            logits_last = logits[:, -1, :]          # (B, vocab_size)
            probs = F.softmax(logits_last, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, idx_next), dim=1)
        return index


# ───────────────────────────────────────────────
# Instantiate model and move to device
# ───────────────────────────────────────────────
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

model = GPTLanguageModel(vocab_size).to(device)
m = model


Using device: cuda


In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()  # switch to eval mode (no dropout, etc.)

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)  # keep on same device

        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss  # keep as tensor, not .item()

        # Move mean to CPU only when needed
        out[split] = losses.mean().detach().cpu()

    model.train()  # back to training mode
    return out


In [17]:
import torch
import torch._dynamo
import pickle

# ───────────────────────────────
# Device setup
# ───────────────────────────────
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True              # best kernel for fixed shapes
torch.set_float32_matmul_precision("high")         # enable fast TF32 on Ampere+
torch._dynamo.config.suppress_errors = True        # avoid torch.compile crashes

# ───────────────────────────────
# Model setup (safe compile)
# ───────────────────────────────

try:
    model = torch.compile(model)
    print("✅ Using torch.compile() optimized mode.")
except Exception as e:
    print(f"⚠️ torch.compile() failed ({e}); running in eager mode.")

m = model  # keep alias consistent

# ───────────────────────────────
# Data batching
# ───────────────────────────────
def get_batch(split):
    data = get_random_chunk(split)
    if len(data) <= block_size + 1:
        raise ValueError("Random chunk too small for requested block_size.")
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,), device=device)
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + 1 + block_size] for i in ix])
    return x.to(device, non_blocking=True), y.to(device, non_blocking=True)

# ───────────────────────────────
# Optimizer and AMP (mixed precision)
# ───────────────────────────────
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# ✅ New API for AMP GradScaler (no deprecation warning)
scaler = torch.amp.GradScaler("cuda", enabled=(device == "cuda"))

# ───────────────────────────────
# Training loop
# ───────────────────────────────
for iter in range(max_iters):

    # Evaluate every 100 steps instead of every step
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"Step {iter:5d} | "
            f"train loss {losses['train'].item():.3f} | "
            f"val loss {losses['val'].item():.3f}"
        )

    xb, yb = get_batch("train")

    # Mixed precision forward/backward
    with torch.autocast(device_type=device, dtype=torch.float16, enabled=(device == "cuda")):
        logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    scaler.scale(loss).backward()

    # Clip gradients for stability
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # Step optimizer safely
    scaler.step(optimizer)
    scaler.update()

print(f"✅ Training complete. Final loss: {loss.item():.4f}")

# ───────────────────────────────
# Save model
# ───────────────────────────────
torch.save(model.state_dict(), "model-01.pt")
print("💾 Model saved to model-01.pt")


Using device: cuda
✅ Using torch.compile() optimized mode.


W1028 10:27:04.928000 3244 site-packages\torch\_inductor\utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233] WON'T CONVERT forward C:\Users\jeeva\AppData\Local\Temp\ipykernel_3244\3040638187.py line 102 
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233] due to: 
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233] Traceback (most recent call last):
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233]   File "c:\Users\jeeva\.conda\envs\LLM\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1164, in __call__
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233]     result = self._inner_convert(
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\convert_frame.py:1233]         frame, cache_entry, hooks, frame_state, skip=skip + 1
W1028 10:27:06.325000 3244 site-packages\torch\_dynamo\conver

Step     0 | train loss 4.490 | val loss 4.490
Step   100 | train loss 2.477 | val loss 2.470
Step   200 | train loss 2.364 | val loss 2.385
Step   300 | train loss 2.257 | val loss 2.258
Step   400 | train loss 2.077 | val loss 2.079
Step   500 | train loss 1.894 | val loss 1.892
Step   600 | train loss 1.745 | val loss 1.748
Step   700 | train loss 1.644 | val loss 1.641
Step   800 | train loss 1.552 | val loss 1.555
Step   900 | train loss 1.480 | val loss 1.487
Step  1000 | train loss 1.446 | val loss 1.449
Step  1100 | train loss 1.391 | val loss 1.399
Step  1200 | train loss 1.341 | val loss 1.354
Step  1300 | train loss 1.316 | val loss 1.306
Step  1400 | train loss 1.289 | val loss 1.277
Step  1500 | train loss 1.261 | val loss 1.257
Step  1600 | train loss 1.217 | val loss 1.220
Step  1700 | train loss 1.198 | val loss 1.193
Step  1800 | train loss 1.154 | val loss 1.155
Step  1900 | train loss 1.141 | val loss 1.144
Step  2000 | train loss 1.101 | val loss 1.109
Step  2100 | 

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.memory_allocated()/1e9, "GB allocated")
print(torch.cuda.memory_reserved()/1e9, "GB reserved")

True
NVIDIA GeForce RTX 3050
0.101321728 GB allocated
0.704643072 GB reserved


In [18]:
prompt = "Hello! Where the hell are you?"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Where the hell are you?"

"That is true, Mr Uncle Henry, I am make my peor hat in this room for man,
but noth charms are of


In [19]:
prompt = "Hello! What can you do for me? \n"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! What can you do for me? 
"Never mind happen, Eureka, meekly."

"I'm very hucknow," but I 'm examined the Sawhorse. As a the G


In [20]:
prompt = "Can you write better prompt than me? \n"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Can you write better prompt than me? 
On either hush of you," said the Sorcerer.

"We don't have to prove ite a straight, and if from
they
